# Hello
I would like to present my very first data science project. Please do not be intimidated by its magnificence.
This is a simple food recommendation project. It gives ideas of dishes basing on your favourite food.
This is first project version, which works on a "dead" dataset. In other version the project will base on API and connect with website to search for recipes.

## Libraries and knowledge

In [ ]:
import os
import numpy as np # linear algebra and arrays
import pandas as pd # dataframes and stuff
import matplotlib.pyplot as plt # plots
%matplotlib inline
import seaborn as sns # more plots
import statistics as st # distributions
import scipy as sp # pivot engineering

#ML model
from sklearn.metrics.pairwise import cosine_similarity

## Data import and first glimpse into it

### Users
u - UserID
techniques - Cooking techniques encountered by user
items - Recipes interacted with, in order
n_items - Number of recipes reviewed
ratings - Ratings given to each recipe encountered by this user
n_ratings - Number of ratings in total

In [ ]:
users_pp = pd.read_csv('archive/PP_users.csv')
users_pp.info()
users_pp.head()

### Recipes - preprocessed data
id - Recipe ID
i - Recipe ID mapped to contiguous integers from 0
name_tokens - BPE-tokenized recipe name
ingredient_tokens - BPE-tokenized ingredients list (list of lists)
steps_tokens - BPE-tokenized steps
techniques - List of techniques used in recipe
calorie_level - Calorie level in ascending order
ingredients_ids - IDs of ingredients in recipe

In [ ]:
recipes_pp = pd.read_csv('archive/PP_recipes.csv')
recipes_pp.info()
recipes_pp.head()

### Recipes - raw data
name - Recipe name
id - Recipe ID
minutes - Minutes to prepare recipe
contributor_id - User ID who submitted this recipe
submitted - Date recipe was submitted
tags - Food.com tags for recipe
nutrition - Nutrition information (calories (#), total fat (PDV), sugar (PDV) , sodium (PDV) , protein (PDV) , saturated fat
steps - Text for recipe steps, in order
description - User-provided description

In [ ]:
recipes_raw = pd.read_csv('archive/RAW_recipes.csv')
recipes_raw.info()
recipes_raw.head()

### Interactions
user_id - User ID
recipe_id - Recipe ID
date - Date of interaction
rating - Rating given
review - Review text

In [21]:
interactions_raw = pd.read_csv('archive/RAW_interactions.csv')
interactions_raw.dropna()
interactions_raw.info()
interactions_raw.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1132367 entries, 0 to 1132366
Data columns (total 5 columns):
 #   Column     Non-Null Count    Dtype 
---  ------     --------------    ----- 
 0   user_id    1132367 non-null  int64 
 1   recipe_id  1132367 non-null  int64 
 2   date       1132367 non-null  object
 3   rating     1132367 non-null  int64 
 4   review     1132198 non-null  object
dtypes: int64(3), object(2)
memory usage: 43.2+ MB


,user_id,recipe_id,date,rating,review
0,38094,40893,2003-02-17,4,Great with a salad. Cooked on top of stove for...
1,1293707,40893,2011-12-21,5,"So simple, so delicious! Great for chilly fall..."
2,8937,44394,2002-12-01,4,This worked very well and is EASY. I used not...
3,126440,85009,2010-02-27,5,I made the Mexican topping and took it to bunk...
4,57222,85009,2011-10-01,5,"Made the cheddar bacon topping, adding a sprin..."


## Data analysis

### Most often rated recipes

I am gonna do it using lists, where list indexes will be recipe ids. This requires creating lists longer by one than maximum recipe_id, so that recipe_id can directly match list index.
Having it done I will create a structured array as in formula:
[(recipe_id, interactions_count, average_rating), ...]

In [267]:
# Create a DataFrame to combine recipe id, its name, number of interactions and average rating

# Count interactions
interactions_count_ser = interactions_raw.value_counts('recipe_id')
interactions_count_ser.name = 'interactions_count'

# Calculate average rate, change column name and extend the DataFrame
avg_ratings_df = interactions_raw[['recipe_id', 'rating']].groupby('recipe_id').mean()
avg_ratings_df.rename(columns = {'rating': 'avg_rating'}, inplace = True)
recipes_ratings_df = avg_ratings_df.join(interactions_count_ser, on = 'recipe_id')

# Add recipes names
recipes_names = pd.DataFrame(data = recipes_raw[['id', 'name']])
recipes_names.set_index('id', inplace = True)
recipes_ratings_df = recipes_ratings_df.merge(recipes_names, left_index = True, right_index = True)

recipes_ratings_df.sort_values(by = 'interactions_count', ascending = False, inplace = True)
recipes_ratings_df.head()

,avg_rating,interactions_count,name
2886,4.185989,1613,best banana bread
27208,4.288570,1601,to die for crock pot roast
89204,4.220393,1579,crock pot chicken with black beans cream cheese
39087,4.541436,1448,creamy cajun chicken pasta
67256,4.329047,1322,best ever banana cake with cream cheese frosting


In [ ]:
# Create lists for interactions count and sum of rates
interactions_count = [0] * (max(recipes_raw['id'].values) + 1)
ratings_sum = [0] * (max(recipes_raw['id'].values) + 1)

# Fill the lists
for row_index, row in interactions_raw.iterrows():
    recipe_id = interactions_raw.loc[row_index]['recipe_id']
    rating = interactions_raw.loc[row_index]['rating']
    interactions_count[recipe_id] += 1
    ratings_sum[recipe_id] += rating

In [ ]:
# Create a structured array to store the recipes stats
array_structure = [('recipe_id', int), ('interactions_count', int), ('avg_rating', float)]
array_values = []

for recipe_id in recipes_raw['id'].values:
    interactions_number = interactions_count[recipe_id]
    avg_rating = ratings_sum[recipe_id] / interactions_count[recipe_id]
    array_values.append((recipe_id, interactions_number, avg_rating))
recipe_stats = np.array(array_values, dtype = array_structure)

In [31]:
# Find 10 most often rated recipes
most_often_recipes = recipe_stats.copy()
most_often_recipes.sort(order = 'interactions_count')
most_often_recipes = most_often_recipes[::-1]

for i in range(10, 0, -1):
    print('Position ', i, ':', sep = '')
    recipe_id = most_often_recipes[i - 1]['recipe_id']
    name = recipes_raw.loc[recipes_raw['id'] == recipe_id]['name'].to_string(index= False)
    print(name.upper())
    print('Interactions:  ', most_often_recipes[i - 1]['interactions_count'])
    avg_rating = most_often_recipes[i - 1]['avg_rating']
    print(f'Average rating: {avg_rating:1.2f} / 5\n')

Position 10:
JAPANESE MUM S CHICKEN
Interactions:   904
Average rating: 4.40 / 5

Position 9:
KITTENCAL S ITALIAN MELT IN YOUR MOUTH MEATBALLS
Interactions:   997
Average rating: 4.71 / 5

Position 8:
WHATEVER FLOATS YOUR BOAT  BROWNIES
Interactions:   1220
Average rating: 4.53 / 5

Position 7:
JO MAMA S WORLD FAMOUS SPAGHETTI
Interactions:   1234
Average rating: 4.42 / 5

Position 6:
YES  VIRGINIA THERE IS A GREAT MEATLOAF
Interactions:   1305
Average rating: 4.21 / 5

Position 5:
BEST EVER BANANA CAKE WITH CREAM CHEESE FROSTING
Interactions:   1322
Average rating: 4.33 / 5

Position 4:
CREAMY CAJUN CHICKEN PASTA
Interactions:   1448
Average rating: 4.54 / 5

Position 3:
CROCK POT CHICKEN WITH BLACK BEANS   CREAM CHEESE
Interactions:   1579
Average rating: 4.22 / 5

Position 2:
TO DIE FOR CROCK POT ROAST
Interactions:   1601
Average rating: 4.29 / 5

Position 1:
BEST BANANA BREAD
Interactions:   1613
Average rating: 4.19 / 5



### Highest rated recipes
At this point I would like to find recipes with highest average rating, but this would probably return a lot of recipes with just one, 5-stars mark.

In [ ]:
# Find 10 top rated recipes

top_rated_recipes = recipe_stats.copy()
top_rated_recipes.sort(order = 'avg_rating')
top_rated_recipes = top_rated_recipes[::-1]

for i in range(10, 0, -1):
    print('Position ', i, ':', sep = '')
    recipe_id = top_rated_recipes[i - 1]['recipe_id']
    name = recipes_raw.loc[recipes_raw['id'] == recipe_id]['name'].to_string(index= False)
    print(name.upper())
    print('Interactions:  ', top_rated_recipes[i - 1]['interactions_count'])
    avg_rating = top_rated_recipes[i - 1]['avg_rating']
    print(f'Average rating: {avg_rating:1.2f} / 5\n')

We got exactly the expected result. So let's make the results more objective and let's set a threshold below which we will not consider results as valid. This could be some constant value (e.g. 20 or 50 ratings), but we want it to be prone to overall ratings number distribution. So let's check what does this distribution look like.

In [ ]:
# Create distribution plot

sns.kdeplot(data = recipe_stats['interactions_count'], log_scale = True, fill = True, cbar = True)
plt.title('Interactions number distribution')
plt.xlabel('Number of interactions (logarithmic scale)')

In [ ]:
# Print some statistical values

data = recipe_stats['interactions_count']
print('Number of recipes:', len(data))
print('Median:    ', st.median(data))
print('Quantiles: ', st.quantiles(data, n = 10))
print('Mean value:', st.mean(data))

After having a look at the chart above and statistical data I decided to set the first threshold as >= 10 interactions, which will cut off majority of lowest popular recipes.

In [ ]:
# Cut off recipes with less than 10 interactions and check how it worked

top_rated_recipes = recipe_stats.copy()
top_rated_recipes.sort(order = 'avg_rating')
top_rated_recipes = top_rated_recipes[::-1]

print('Number of recipes included:', len(top_rated_recipes))
print('5 first elements, second column is number of interactions:')
print(top_rated_recipes[0:5])

print('\n', '*'*50, '\n')

filter_array = []
for element in top_rated_recipes:
    if element['interactions_count'] >= 10:
        filter_array.append(True)
    else:
        filter_array.append(False)
top_rated_recipes = top_rated_recipes[filter_array]
print('Number of recipes included:', len(top_rated_recipes))
print('5 first elements, second column is number of interactions:')
print(top_rated_recipes[0:5])

As we can see, setting minimum interactions count to just 10 interactions let us filter 90% of recipes. Let's check the distribution and statistical data now:

In [ ]:
# Create - again - distribution plot

sns.kdeplot(data = top_rated_recipes['interactions_count'], log_scale = True, fill = True, cbar = True)
plt.title('Interactions number distribution')
plt.xlabel('Number of interactions (logarithmic scale)')

In [ ]:
# Print - again - some statistical values

data = top_rated_recipes['interactions_count']
print('Number of recipes:', len(data))
print('Median:    ', st.median(data))
print('Quantiles: ', st.quantiles(data, n=10))
print('Mean value:', st.mean(data))

For now we can consider it as a decent outcome. Let's get our desired top recipe list!

In [ ]:
# Print top rated recipes

for i in range(10, 0, -1):
    print('Position ', i, ':', sep = '')
    recipe_id = top_rated_recipes[i - 1]['recipe_id']
    name = recipes_raw.loc[recipes_raw['id'] == recipe_id]['name'].to_string(index= False)
    print(name.upper())
    print('Interactions:  ', top_rated_recipes[i - 1]['interactions_count'])
    avg_rating = top_rated_recipes[i - 1]['avg_rating']
    print(f'Average rating: {avg_rating:1.2f} / 5\n')

Ok, sorry for that - that's again a list of only-5-stars recipes. So let's filter it once again and check how it looks for at least 100 interactions:

<again?_again_meme>

In [ ]:
# Cut off recipes with less than 100 interactions and check how it worked

print('Number of recipes included:', len(top_rated_recipes))
print('5 first elements, second column is number of interactions:')
print(top_rated_recipes[0:5])

print('\n', '*'*50, '\n')

filter_array = []
for element in top_rated_recipes:
    if element['interactions_count'] >= 100:
        filter_array.append(True)
    else:
        filter_array.append(False)
top_rated_recipes = top_rated_recipes[filter_array]
print('Number of recipes included:', len(top_rated_recipes))
print('5 first elements, second column is number of interactions:')
print(top_rated_recipes[0:5])

In [ ]:
# Create - yes, again - distribution plot

sns.kdeplot(data = top_rated_recipes['interactions_count'], log_scale = True, fill = True, cbar = True)
plt.title('Interactions number distribution')
plt.xlabel('Number of interactions (logarithmic scale)')

In [ ]:
# Print - yes, again - some statistical values

data = top_rated_recipes['interactions_count']
print('Number of recipes:', len(data))
print('Median:    ', st.median(data))
print('Quantiles: ', st.quantiles(data, n=10))
print('Mean value:', st.mean(data))

In [ ]:
# Print top rated recipes

for i in range(10, 0, -1):
    print('Position ', i, ':', sep = '')
    recipe_id = top_rated_recipes[i - 1]['recipe_id']
    name = recipes_raw.loc[recipes_raw['id'] == recipe_id]['name'].to_string(index= False)
    print(name.upper())
    print('Interactions:  ', top_rated_recipes[i - 1]['interactions_count'])
    avg_rating = top_rated_recipes[i - 1]['avg_rating']
    print(f'Average rating: {avg_rating:1.2f} / 5\n')

Now we got a really nice top recipes list, which means we can head to the supermarket for some tex-mex ingredients :)

## Recommendation tools

### Creating DataFrame for pivot table

We will have to limit size of the table, because it would get too big to be processed by Pandas. To do so we will delete users with only one interaction and recipes also with only one interaction. These will be our steps:
 1. Create a new DataFrame to aggregate interactions count for each recipe.

In [75]:
# Step 1
min_recipe_interactions = 10
recipes_valid = interactions_raw['recipe_id'].value_counts()
recipes_valid = recipes_valid[recipes_valid.ge(min_recipe_interactions)]
interactions_valid = interactions_raw[interactions_raw['recipe_id'].isin(recipes_valid.index)]

min_user_interactions = 3
users_valid = interactions_raw['user_id'].value_counts()
users_valid = users_valid[users_valid.ge(min_user_interactions)]
interactions_valid = interactions_valid[interactions_valid['user_id'].isin(users_valid.index)]


In [76]:
# Create a new DataFrame combining user id, recipe name and its rating

rated_recipes = interactions_valid.merge(recipes_raw, left_on = 'recipe_id', right_on = 'id')
rated_recipes = rated_recipes[['user_id', 'name', 'rating']]
print(len(rated_recipes))
rated_recipes.head()

473751


,user_id,name,rating
0,56680,potato crab chowder,5
1,183565,potato crab chowder,5
2,101823,potato crab chowder,5
3,349752,potato crab chowder,5
4,446143,potato crab chowder,4


In [77]:
# Create pivot table to gather users and their ratings in one table

pivot = pd.pivot_table(rated_recipes, values = 'rating', index = 'user_id', columns = 'name')
pivot.head()

name,1 00 tangy chicken recipe,1 1 1 tempura batter,1 2 3 4 tater tot casserole,1 2 3 4 cake,1 2 3 4 cake with caramel icing,1 2 3 apple crisp,1 2 3 black bean salsa dip,1 cup of coffee pork roast,1 hour ham and bean soup,1 minute cake,...,zucchini spoon bread,zucchini squash rice casserole,zucchini stuffing,zucchini tart with gruyere cheese and herbs,zucchini they will eat,zucchini with summer herbs,zucotte,zuppa di pesce cioppino or fish stew,zuppa toscana soup olive garden clone,zurie s overnight no knead bread
user_id,,,,,,,,,,,,,,,,,,,,,
1533,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1535,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1634,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1676,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1773,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Engineering pivot table

We will do it in following steps:
1. Value normalization
2. Filling NaN values with 0
3. Transposing the pivot for the next step
4. Dropping columns with values of 0
5. Using scipy package to convert to sparse matrix format for the similarity computation


In [79]:
# Step 1
pivot_n = pivot.apply(lambda x: (x - np.mean(x)) / (np. max(x) - np.min(x)), axis = 1)
# Step 2
pivot_n.fillna(0, inplace = True)
# Step 3
pivot_n = pivot_n.T
# Step 4
pivot_n = pivot_n.loc[:, (pivot_n != 0).any(axis = 0)]
# Step 5
pivot_sparse = sp.sparse.csr_matrix(pivot_n.values)

## Cosine similarity model

This method measures similarity of two vectors by calculating angle between them.

In [82]:
# Model based on recipe similarity
recipe_similarity = cosine_similarity(pivot_sparse)

# DataFrame of recipes similarity
recipe_similarity_df = pd.DataFrame(recipe_similarity, index = pivot_n.index, columns = pivot_n.index)

In [153]:
def recommend_recipe(liked_recipe):
    recipe_similarity_df.sort_values(by = liked_recipe, ascending = False, inplace = True)
    print('Recommmended, because you liked:', liked_recipe)
    for number in range(1, 6):
        recommended_recipe = recipe_similarity_df.index[number]
        match = float(recipe_similarity_df.iloc[number][liked_recipe]) * 100
        print(f'Top {number}: {recommended_recipe}, match: {match:2.2f} %')

To properly use our recommendation method an exact base recipe name must be given as an argument. So let's prepare an easy searching tool that finds recipes names that contain given phrase. Then, let's check results of the final project!

In [176]:
def find_recipe(name):
    found_recipes = []
    for element in recipe_similarity_df.index:
        if name in element:
            found_recipes.append(element)

    for element in found_recipes:
        print(f'{element} - Rating: dopisać')

In [190]:
find_recipe('chicken')

japanese crispy chicken - Rating: dopisać
baked salsa chicken breast - Rating: dopisać
japanese mum s chicken - Rating: dopisać
mediterranean chicken paella - Rating: dopisać
sandra s chicken - Rating: dopisać
cashew chicken curry - Rating: dopisać
crock pot chicken and sausage gumbo with shrimp - Rating: dopisać
lemon chicken just like take out - Rating: dopisać
cheat n  eat vietnamese chicken soup - Rating: dopisać
thai chicken cakes with sweet chilli sauce - Rating: dopisać
huli huli chicken or ribs - Rating: dopisać
pesto chicken - Rating: dopisać
ginger chicken noodle soup  crock pot - Rating: dopisać
caramelized chicken wings - Rating: dopisać
easy crock pot teriyaki chicken - Rating: dopisać
spicy honey orange chicken - Rating: dopisać
chicken bombay  oamc - Rating: dopisać
korean spicy chicken and potato  tak toritang - Rating: dopisać
garlic ginger chicken strips - Rating: dopisać
melt in your mouth barbecued chicken - Rating: dopisać
ranch chicken thighs - Rating: dopisać
emp

In [189]:
recommend_recipe('japanese crispy chicken')

Recommmended, because you liked: japanese crispy chicken
Top 1: asian inspired tuna cakes, match: 82.64 %
Top 2: carbonara, match: 63.02 %
Top 3: red wine steak and mushrooms, match: 60.54 %
Top 4: t g i  fridays jack daniels grill glaze, match: 59.10 %
Top 5: sheila s best salmon, match: 52.99 %
